In [ ]:
kubectl config use-context kubernetes-admin@kubernetes-arc

$ENV:AZDATA_USERNAME='arcadmin'
$ENV:AZDATA_PASSWORD='S0methingS@Str0ng!'

azdata login --endpoint https://192.168.13.131:30080 --namespace arc
azdata arc postgres server create -n postgres01 --workers 0 --dev
azdata arc sql mi create -n sqldemo01 --dev

 **First up, let's log into our Azure Arc Data Controller using the Azure Data CLI ( `azdata` )**

In [ ]:
$ENV:AZDATA_USERNAME='arcadmin'
$ENV:AZDATA_PASSWORD='S0methingS@Str0ng!'

azdata login --endpoint https://192.168.13.131:30080 --namespace arc

**As with any CLI tool, your first stop should be the help :) Get familiar with the syntax and options available.**

In [ ]:
azdata arc --help

**I've  already deployed one Managed Instance and one PostgreSQL Hyperscale instance into our Arc Data Services on premises environment. To get connection information we can use `azdata`.**

In [ ]:
azdata arc sql --help
azdata arc sql mi list

In [ ]:
azdata arc postgres endpoint list --name postgres01 -o table

**Services in Arc enabled Data Services are implemented as Kubernetes Services.**  
**We can use Kubernetes native tooling to interact with our Arc Enabled Data Services Instances and Configurations.**  
**By default you get two Service Endpoints per Service, internal (ClusterIP) and External (NodePort)**

In [ ]:
kubectl get services --namespace arc 

**Let's get the Port (NodePort) for our deployed Managed Instance which is backed by the Service `sqldemo01-external-svc`**

In [ ]:
$NodePort=kubectl get services --namespace arc sqldemo01-external-svc -o jsonpath='{ .spec.ports[0].nodePort }'
sqlcmd -S 192.168.13.131,$NodePort -U arcadmin -P 'S0methingS@Str0ng!' -Q 'SELECT @@VERSION'

**Let's begin the process of deploying a Managed Instance using `azdata`. We'll start with checking out the help first...and check out some of the options available to us.**

-   **Limits -** an upper limit for access to resources, CPU and Memory. 
-   **Requests -** guaranteed access to resources, CPU or Memory.
-   **StorageClasses -** dynamically provisioned cluster storage attached into the Pod for Persistent Storage. By default, you will get a Data (includes transaction log) and a Log Persistent Volume.

In disconnected mode, you can also deploy with Azure Data Studio.  GitOps, Portal, Azure CLI and PowerShell experiences are all on the way.

In [ ]:
azdata arc sql mi create --help

**We can create one with minimal options which will get you a Managed Instance with no CPU or Memory limits and Requests of 100m and 100Mi and a single disk from your default StorageClass. Takes about 1 minute with containers pre-pulled.**

In [ ]:
azdata arc sql mi create --name sqldemo02 --dev

**Let's check out our newly deployed Managed Instance. Now we have two unique Service Endpoints.**

In [ ]:
azdata arc sql mi list 

**Now with all of that let's check out some of the management tooling and to do that...let's upload some data into Azure. You can use environment variables to help build automation around these tools. Otherwise the prompts are going to be interactive...asking for things like username, namespace, workspace\_id and so on.**

In [ ]:
$ENV:AZDATA_USERNAME="arcadmin"
$ENV:AZDATA_PASSWORD="S0methingS@Str0ng!"

azdata login --namespace arc


$ENV:WORKSPACE_ID='31f660e3-cb32-4f1c-97f1-e69896eca90f'
$ENV:SPN_CLIENT_ID='a7b906f4-297f-4aaa-821d-a410e398973e'
$ENV:SPN_CLIENT_SECRET='2ec0a112-0d25-4168-81fd-9917f86cabe4'
$ENV:SPN_TENANT_ID='f17cbd44-7697-453e-941c-efe0a4c2d55a'
$ENV:WORKSPACE_SHARED_KEY='TNeKgNLo67T9+8dAN1oDdBzrdwiIdJJ/HD1dwbWRCuWIbyuET/SR/ZpUtkRQvkZxYxrYgB+9niNzXQxAZMQdkg=='
$ENV:SPN_AUTHORITY='https://login.microsoftonline.com'

azdata arc dc export --type metrics --path metrics.json --force
azdata arc dc upload --path metrics.json

azdata arc dc export --type logs --path logs.json --force
azdata arc dc upload --path logs.json

**Management, Monitoring and Logging** - our implementation of Azure Arc Enabled Data Services is disconnected. The local tools hold the metrics, logging and usage data. One uploaded, you can use tools in Azure to analyze that data. Let's look at the local tools then hop into Azure to look at the tools available there.

1.  Right click Manage to Launch the **Azure Arc Data Controller Dashboard**
    * Click on the Managed Instance **sqldemo01** to launch the management dashboard
    * To view logs on-prem click Kibana Dashboard
        * Set the date range to 15 days ago and search for **master** and **tempdb**
    * To view Metrics on-prem click Grafana Dashboard
        * Review the core SQL Server metrics
        
1. Back on the main **Azure Arc Data Controller Dashboard**, notice the Connection mode of the Data Controller: **Indirect**. Click on **Open in Azure Portal**.    
1. Click on the Resource Group name in our case it's **arc-onprem**
1. Check out the resources in the Portal
    * Click on Metrics, change the **Metric Namespace** to ```sql server``` **change the Metric** to ```Memory Usage```. change the date range to 24 hours.
1. In the Log Analytics workspace and run the following query
    -   ```
         sqlManagedInstances_logs_CL |  where TimeGenerated > ago(7d)  |  order by TimeGenerated desc 
        ```

In [ ]:
azdata arc sql mi delete --name sqldemo02
azdata arc sql mi delete --name sqldemo01
azdata arc postgres server delete --name postgres01
azdata logout

**Everything deployed inside of Azure Arc Data Services is implemented as a Kubernetes Object...let's check out a few.** 

-   **Pods** - running container based applications inside the Kubernetes cluster
-   **Services** - access endpoints to reach the applications running inside our cluster. There can be cluster internal (ClusterIP) and external Service endpoints (NodePort). Can be integrated with external LoadBalancers if needed.
-   **DaemonSets** - ensure a Pod is running on each node in the cluster. Used by the Metrics data collector in Arc, in our case [Telegraf](https://github.com/influxdata/telegraf "https://github.com/influxdata/telegraf").
-   **ReplicaSets** -  ensure a desired number of Pods are running in the cluster.  Primarily used by management software and web ports for logs and metrics (bootstrapper, control, controlwd, logsui, metricsui and mgmtproxy)
-   **StatefulSets** - ensure a desired number of Pods are running in the cluster. But also provide stable naming on the Pods and cluster service discovery via Cluster DNS. Used by controldb, logsdb, metricsdb and also provisioned Managed Instances and Postgres instances. Enables scaling.

In [ ]:
kubectl get all --namespace arc